In [1]:
import os
import pandas as pd
import numpy as np



In [2]:
# Set working directory
#os.chdir(r"insert working directory")


In [3]:
# insert weekly fraction for domestic and industrial
dom_weekly_fraction = {
    "Mon": 1.0,
    "Tue": 1.0,
    "Wed": 1.0,
    "Thu": 1.0,
    "Fri": 1.0,
    "Sat": 0.95,
    "Sun": 0.95
}

ind_weekly_fraction = {
    "Mon": 1.0,
    "Tue": 1.0,
    "Wed": 1.0,
    "Thu": 1.0,
    "Fri": 1.2,
    "Sat": 0.25,
    "Sun": 0.25
}


In [4]:
## Get day from the input file

# Read the CSV file
file_path = "./input/input_file.csv"        # Adjust path as needed
df = pd.read_csv(file_path, parse_dates=["Date"])  # read the data

# Get day of the week
df["Day"] = df["Date"].dt.strftime("%a")           # e.g., 'Mon', 'Tue'

# Map to weekly fraction for domestic
df["dom_weekly_fraction"] = df["Day"].map(dom_weekly_fraction)

# Map to weekly fraction for industrial
df["ind_weekly_fraction"] = df["Day"].map(ind_weekly_fraction)



In [5]:
### Calculate domestic flow
pop = 550000          # number of population in the sewershed
q = 0.28             # domestic wastewater production per population per day (m3/pop.day)

# Calculate Domestic Flow
df["Q_domestic"] = pop * q * df["dom_weekly_fraction"]



In [6]:
### Calculate Industrial flow

Qind_daily = 2400;               # daily industrial wastewater production (m3/d)

# Calculate Industrial Flow
df["Q_industrial"] = Qind_daily * df["ind_weekly_fraction"]


In [7]:
### Calculate rainfall-drived inflow

K_RDI = 1600    # co-efficient to convert rainfall to direct runoff (how much runoff is generated from 1 mm rainfall) 

df["Q_RDI"] = K_RDI * df["Rainfall (mm/day)"]



In [8]:
### Calculate rainfall-induced infiltration (RII)

# set Parameters 
kd = 0.07     # decay constant (/day)
K_inf = 2250  # a scaling parameter representing the infiltration gain (m3/mm)

rain = df["Rainfall (mm/day)"]
n = len(rain)

# Apply Delayed Effect (Exponential Decay) [later part of eqn 8]

delayed_rain = np.zeros(n)
for t in range(n):
    for j in range(t + 1):
        delayed_rain[t] += rain[j] * np.exp(-kd * (t - j))



df["delayed_rain"]= delayed_rain 



# Calculate Qinf 
Q_inf = K_inf *delayed_rain # equation

# Save in dataframe
df["Q_inf"] = Q_inf


## Apply seasonal correction factor using influent data
alpha = 0.3
df["SCF_influent"] =df["seasonal"]**alpha

## Apply seasonal correction factor using water level data
beta = 0.33
df["SCF_WL"] =df["WL"]**beta



df["SCF_total"] = (df["SCF_influent"] + df["SCF_WL"])*0.5

df["Q_inf_SCF"] = df["Q_inf"] * df["SCF_total"]



In [9]:
### Calculate influent and save the file 
df["Influent (m3/day)"] = df["Q_domestic"] + df["Q_industrial"] + df["Q_RDI"]  + df["Q_inf_SCF"] 


In [10]:
### Sewer Transport
kr = 1.05
V = df.loc[df.index[0], "Influent (m3/day)"]

Q_influent = []

for t in range(len(df)):
    Q_in = df.loc[df.index[t], "Influent (m3/day)"]
    Q_out = V / kr
    V += (Q_in - Q_out) 
    Q_influent.append(V)

df["Discharge_sewer m3/day"] = Q_influent

In [11]:

# convert m3/day to MGD
conversion_MGD_2_m3pD = 3785.41; # convert MGD to m3/day

df["Influent (MGD)"] = df["Discharge_sewer m3/day"] /conversion_MGD_2_m3pD

df.to_csv("output/Influent.csv", index=False)

